# Регуляризация линейной регрессии

*Разминка*: Всегда ли решение задачи линейной регрессии единственно?


*Вопрос*: А если все же линейной зависимости нет? Подсказка: решение задачи регрессии имеет вид. $(X^TX)^{-1}X^Ty$.
 

*Вопрос*: К чему это приведет?


*Вопрос:* Какие еще последствия у мультиколлинеарности?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.typing import NDArray
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import (
    HuberRegressor,
    LinearRegression,
    RANSACRegressor,
    TheilSenRegressor,
)
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import load_diabetes, fetch_openml,load_iris,fetch_california_housing

from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata

np.random.seed(42)

# Переобучение
В машинном обучении часто говорят об *обобщающей способности модели*, то есть о способности модели работать на новых тестовых данных хорошо. Если модель будет идеально предсказывать выборку, на которой она обучалась, но при этом просто ее запомнит, не \"вытащив\" из данных никакой закономерности, от нее будет мало толку. Такую модель называют *переобученной*: она слишком подстроилась под обучающие примеры, не выявив никакой полезной закономерности, которая позволила бы ей совершать хорошие предсказания на данных, которые она ранее не видела.\n",

Рассмотрим следующий пример, на котором будет хорошо видно, что значит переобучение модели. Для этого нам понадобится сгенерировать синтетические данные. Рассмотрим зависимость $y = cos(1.5\pi x)$, 
 $y $ — целевая переменная, а $x$ - объект (в нашем случае это число от 0 до 1). В жизни мы наблюдаем какое-то конечное количество пар объект-таргет, поэтому смоделируем это, взяв 30 случайных точек  $x_i, y_i$  в отрезке $[0, 1]$.

В реальной жизни целевая переменная может быть зашумленной, смоделируем это, зашумив значение функции нормальным шумом: $\tilde{y}_i = y(x_i) + \mathcal{N}(0, 0.01)$.

Попытаемся обучить три разных линейных модели: признаки для первой - $\{x\}$, для второй - $\{x, x^2, x^3, x^4\}$, для третьей - $\{x, \dots, x^{20}\}$.
    

In [ ]:
np.random.seed(36)
x = np.linspace(0, 1, 100)
y = np.cos(1.5 * np.pi * x)

x_objects = np.random.uniform(0, 1, size=30)
y_objects = np.cos(1.5 * np.pi * x_objects) + np.random.normal(scale=0.1, size=x_objects.shape)

from sklearn.preprocessing import PolynomialFeatures
fig, ax = plt.subplots(figsize=(16, 4), ncols=3)
for i, degree in enumerate([1, 4, 20]):
    X_objects = PolynomialFeatures(degree, include_bias=False).fit_transform(x_objects[:, None])
    X = # YOUR CODE: Transform x (to get nice line of prediction)
    model = LinearRegression().fit(X_objects, y_objects)
    y_pred = model.predict(X)
    ax[i].plot(x, y, label="Real function")
    ax[i].scatter(x_objects, y_objects, label="Data")
    ax[i].plot(x, y_pred, label="Prediction")
    if i == 0:
        axs[i].legend()
    ax[i].set_title("Degree = %d" % degree)
    ax[i].set_xlabel("$x$")
    ax[i].set_ylabel("$f(x)$")
    ax[i].set_ylim(-2, 2)

**Вопрос**: Почему первая модель получилась плохой, а третья переобучилась?

Чтобы избежать переобучения, модель регуляризуют. На этом занятии мы как раз рассмотрим различные методы регуляризации линейных моделей.

Но, прежде, чем начать, создадим свою версию линейной модели, которую можно инициализировать готовыми весами и не обучать на данных. Зачем нам это понадобится? Узнаем чуть позже.
Итак, создайте класс линейной модели, которую можно будет инициализировать заданными коэффициентами. Она должна реализовывать интерфейс модели sklearn. Можете отнаследоваться от стандартной линейной модели.

In [ ]:
class LinearPredictionModel(LinearRegression):
    def __init__(self, coef=None, intercept=None):
        if coef is not None:
            coef = # make a numpy array from coefficients
            if intercept is None:
                intercept = np.zeros(coef.shape[0])
            else:
                intercept = np.array(intercept)
            assert coef.shape[0] == intercept.shape[0]
        else:
            if intercept is not None:
                raise ValueError("Provide coef only or both coef and intercept")
        self.intercept_ = intercept
        self.coef_ = coef


Для первых экспериментов также создадим случайный набор данных, где x имеет 2 признака. Можете обучить свою или стандартную модель на этих данных.

In [ ]:
x = np.random.randn(5,2)
real_alpha = 1.5
real_beta = 2.
y = real_alpha * x[:, 0] + real_beta * x[:, 1] + np.random.normal(loc=0., scale=0.2)
model = LinearRegression().fit(x, y)
print(f"coefficients: {model.coef_}, intercept: {model.intercept_}")

Создадим функцию, которая будет сохранять значение ошибки для разных значений параметров. Функция должна возвращать датафрейм со столбцами `["a", "b", "error"]`.

In [ ]:
def get_errors_table(
    alphas: list[float] | NDArray, betas: list[float] | NDArray, x: NDArray, y: NDArray
) -> pd.DataFrame:
    result = pd.DataFrame(columns=["a", "b", "error"])
    for alpha in alphas:
        for beta in betas:
            res = {"a": alpha, "b": beta}
            model = LinearPredictionModel(coef=[[alpha, beta]], intercept=[[0]])
            res["error"] = mean_squared_error(model.predict(x), y)
            result = pd.concat([result, pd.DataFrame(res, index=[0])], axis=0, ignore_index=True)
    return result
            

Зададим сетку параметров:

In [ ]:
alphas = np.linspace(real_alpha - 2, real_alpha + 2, 101)
betas = np.linspace(real_beta - 2, real_beta + 2, 101)


In [ ]:
alphas

In [ ]:
df = get_errors_table(alphas, betas, x, y)

Нарисуйте поверхность функции потерь в 3d. Для отрисовки удобнее всего воспользовать функцией `plot_trisurf`. Для создания графика можно воспользоваться `fig.add_subplot(subplot_pos (int: n_hor|n_vert|position), projection='3d')`

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
# Один способ нарисовать 3d график
x1 = np.linspace(df['a'].min(), df['a'].max(), len(df['a'].unique()))
y1 = np.linspace(df['b'].min(), df['b'].max(), len(df['b'].unique()))
x2, y2 = np.meshgrid(x1, y1) 

z2 = griddata((df['a'], df['b']), df['error'], (x2, y2), method='cubic')
ax.plot_trisurf(df["a"], df["b"], df["error"], alpha=0.5)
plt.show()

Также можно нарисовать линии уровня для этой поверхности. Можно воспользоваться функцией `ax.tricontour`.

In [ ]:
fig, ax = plt.subplots()
CS = ax.tricontour(df['a'], df['b'], df['error'], levels=15 )

Отлично! Теперь можно переходить к вопросам регуляризации. 

Вместо исходной задачи можно решать другую, с учетом регуляризации:
\begin{equation}
\min_w L(f, X, y) = \min_w(|X w - y|_2^2 + \lambda |w|^k_k ),
\end{equation}
где 
\begin{equation}
|w|^2_2 = w^2_1 + \ldots + w^2_n
\end{equation}
\begin{equation}
|w|_1^1 = \vert w_1 \vert + \ldots + \vert w_n \vert
\end{equation}

Коэффициент $\lambda$ - коэффициент регуляризации, его часто подбирают по логарифмической шкале.


Вопрос: Нужно ли регуляризовать отступ?

Решение новой задачи (L2) имеет вид 
\begin{equation} 
w = (X^TX + \lambda I)^{-1}X^Ty
\end{equation}
В случае плохо обусловленной матрицы можно сразу увидеть эффект от регуляризации. Какой?
Градиент же имеет вид
\begin{equation}
\nabla_wL(f_w, X, y) = 2X^T(Xw - y) + 2\lambda w
\end{equation}

Вопрос: Какой вариант градиентного спуска правильный для батча размера N:
\begin{equation}
(a) w_i\mapsto w_i - 2\alpha(\langle w, x_j\rangle - y_j)x_{ji} - \frac{2\alpha\lambda}N w_i,\quad i=1,\ldots,D
\end{equation}
\begin{equation}
(b) w_i\mapsto w_i - 2\alpha(\langle w, x_j\rangle - y_j)x_{ji} - 2\alpha\lambda w_i,\quad i=1,\ldots,D
\end{equation}
\begin{equation}
(c) w_i\mapsto w_i - 2\alpha(\langle w, x_j\rangle - y_j)x_{ji} - 2\lambda N w_i,\quad i=1,\ldots D
\end{equation}


Вопрос: Как выглядит градиент в случае L1- регуляризации? 

Вопрос: А как быть в точке, где градиента не существует?

Вопрос: Можно ли применить регуляризацию для другой функции потерь?

Теперь попробуем геометрически отрисовать, как выглядят составляющие функции потерь после регуляризации. Для начала реализуем функции рассчета нормы.

In [ ]:
import math
def l1_norm(vector):
    return # Make a l1 norm regularizer for vector

def l2_norm(vector):
    return # Make a l2 norm regularizer for vector

def p_norm(vector, p: float=1.):
    return  # Make a lp norm regularizer for vector

Зададим пространство параметров, как и ранее, но построим таблицу норм весов вместо ошибок.

In [ ]:
alphas = np.linspace( - 2, real_alpha + 2, 101)
betas = np.linspace( - 2, real_beta + 2, 101)

def get_norm_plot(alphas, betas, norm_fn):
    result = pd.DataFrame(columns=["a", "b", "norm"])
    for alpha in alphas:
        for beta in betas:
            res = {"a": alpha, "b": beta}
            res["norm"] = norm_fn(np.array([alpha, beta]))
            result = pd.concat([result, pd.DataFrame(res, index=[0])], axis=0, ignore_index=True)
    return result
            

In [ ]:
df_norm = get_norm_plot(alphas, betas, l1_norm)

In [ ]:

fig = plt.figure(figsize=(12,8))
ax_1 = fig.add_subplot(121, projection='3d')
ax_2 = fig.add_subplot(122)

# Interpolate unstructured D-dimensional data.
ax_1.plot_trisurf(df_norm["a"], df_norm["b"], df_norm["norm"], alpha=0.5)
ax_2.tricontour(df_norm['a'], df_norm['b'], df_norm['norm'], levels=15 )
plt.show()

In [ ]:
df_norm = get_norm_plot(alphas, betas, l2_norm)

fig = plt.figure(figsize=(12,8))
ax_1 = fig.add_subplot(121, projection='3d')
ax_2 = fig.add_subplot(122)

# Interpolate unstructured D-dimensional data.
ax_1.plot_trisurf(df_norm["a"], df_norm["b"], df_norm["norm"], alpha=0.5)
ax_2.tricontour(df_norm['a'], df_norm['b'], df_norm['norm'], levels=15 )
plt.show()

Вопрос: Что будет с поверхностью, если строить график для регулязированной функции ошибки?

Чтобы проверить свой ответ, построим его. Для этого реализуйте функцию для рассчета новой ошибки (с функцией регуляризации как параметром)

In [ ]:
def get_error_plot(alphas, betas, norm_fn, gamma, x, y):
    result = pd.DataFrame(columns=["a", "b", "error"])
    for alpha in alphas:
        for beta in betas:
            res = {"a": alpha, "b": beta}
            model = # Make a linear model with given coefficients. Use intercept 0 
            res["error"] = mean_squared_error(model.predict(x), y) + gamma * norm_fn(np.array([alpha, beta]))
            result = pd.concat([result, pd.DataFrame(res, index=[0])], axis=0, ignore_index=True)
    return result
            
            

In [ ]:
alphas = np.linspace( - 2, real_alpha + 2, 101)
betas = np.linspace( - 2, real_beta + 2, 101)
df_norm = get_error_plot(alphas, betas, l1_norm, 4., x, y)

fig = plt.figure(figsize=(12,8))
ax_1 = fig.add_subplot(121, projection='3d')
ax_2 = fig.add_subplot(122)

ax_1.plot_trisurf(df_norm["a"], df_norm["b"], df_norm["error"], alpha=0.5)
ax_2.tricontour(df_norm['a'], df_norm['b'], df_norm['error'], levels=15)
plt.show()

In [ ]:
alphas = np.linspace( - 2, real_alpha + 2, 101)
betas = np.linspace( - 2, real_beta + 2, 101)
df_norm = get_error_plot(alphas, betas, l2_norm, 4., x, y)

fig = plt.figure(figsize=(12,8))
ax_1 = fig.add_subplot(121, projection='3d')
ax_2 = fig.add_subplot(122)

ax_1.plot_trisurf(df_norm["a"], df_norm["b"], df_norm["error"], alpha=0.5)
ax_2.tricontour(df_norm['a'], df_norm['b'], df_norm['error'], levels=15)
plt.show()

**Вопрос 1**: Почему большие веса в линейной модели — плохо?

**Вопрос 2**: Почему регуляризовать $w_0$
 — плохая идея?
 
**Вопрос 3**: На что влияет коэффициент $\lambda$? Что будет происходить с моделью, если его
 начать уменьшать? Что будет, если его сделать слишком большим?

## Спарсность при использовании L1

У L1-регрессии есть еще одно замечательное свойство. Оно заключается в том, что оптимальное решение в случае задачи с ограничениями находится в точке пересечения с основной поверхностью. Это будет происходить в углах ромба, где один из весов занулен. В результате, при увеличении размерности, у признаков, не оказывающих влияния на ответы, веса постепенно зануляются.
Таким образом, этот вид регрессии тоже можно использовать для выбора признаков.

*Вопрос:* Может быть задача, при которой спарсность не будет наблюдаться?

Для иллютрации рассмотрим уже знакомый датасет по классификации цифр. Однако, для нужд регрессии сделаем таргет, равный "яркости" написания - суммы всех значений пикселей.

In [ ]:
mnist = fetch_openml('mnist_784', cache=False)

X = mnist["data"].astype('float64')
X.reset_index()
y_c = mnist["target"].astype('int64')
y_c.reset_index()

print("\nNo. of Samples: ", X.shape)
print("No. of Labels: ", y_c.shape)

In [ ]:
y = X.sum(axis=1) + X.iloc[:,0:20].sum(axis=1) * 2 + X.iloc[:,100:400].sum(axis=1) * 1.5

In [ ]:
random_indices = np.random.choice(X.index, 10000, replace=False)
X_rus = X.loc[random_indices]
y_rus = y.loc[random_indices]

X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.2, random_state=0)


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
print(model.coef_[3], model.intercept_)

In [ ]:
sns.kdeplot(model.coef_)

In [ ]:
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso(alpha=1.)
# YOUR CODE: fit lasso
print(lasso.coef_[3], lasso.intercept_)
sns.kdeplot(lasso.coef_)

Увеличим значение регуляризации

In [ ]:
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso(alpha=4.)
# YOUR CODE: fit lasso
print(lasso.coef_[3], lasso.intercept_)
sns.kdeplot(lasso.coef_)

In [ ]:
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso(alpha=20.)
# YOUR CODE: fit lasso
print(lasso.coef_[3], lasso.intercept_)
sns.kdeplot(lasso.coef_)

Как видно, чем больше параметр регуляризации, тем больше весов уходит в ноль.

In [ ]:
from sklearn.linear_model import Lasso, Ridge
model = Ridge()
model.fit(X_train, y_train)
print(model.coef_[3], model.intercept_)
sns.kdeplot(model.coef_)

Посмотрим, будем ли мы наблюдать такой же эффект в случае Ridge регрессии. 

In [ ]:
# Make the same eexperiment with alpha=5

In [ ]:
# Make the same eexperiment with alpha=20

Как видно, в данном случае это не наблюдается. Кстати, это можно понять и по геометрическому представлению L2-регуляризации. 

Какой из видов регуляризации выбрать? 
Часто решение зависит от задачи.

Так, L2-регрессия больше внимания уделяет большим весам, а вот относительно маленькие будут в низком приоритете. Почему?

Также спарсность - очень полезное свойство. Однако иногда L1 регрессия проигрывает L1. Почему?

Кроме того:
- В случае p>n L1 выбирает не более n переменных до насыщения из-за природы задачи выпуклой оптимизации.
- Если существует группа переменных, среди которых попарные корреляции очень высоки, то L1 стремится выбрать только одну переменную из группы, и ей все равно, какую именно
- Для обычных n>p ситуаций, если существуют высокие корреляции между предикторами,эмпирически замечено, что в предсказательной производительности L2 доминирует над L1

## Elastic Net
Эта модель была предложена как способ исправить проблемы первых двух пунктов, но при этом достичь лучших рещультатов, чем Lasso, в целом.
Она устроена на удивление просто, как комбинация первых двух, и практически всегда будет хорошим решением. Единственная проблема в том, что в ее случае приходится тюнить сразу два параметра:
\begin{equation}
\min_w L(f, X, y) = \min_w(|X w - y|_2^2 + \lambda_1 |w|^2_2 + \lambda_2 |w|_1),
\end{equation}

In [ ]:
## Elastic Net
from sklearn.linear_model import ElasticNet
# Make the same eexperiment as for ridge and lasso with alpha=5

В данном случае мы наблюдаем смешанный эффект. 

## Задача

**Задание**. Протестируйте разные методы регуляризации на наборе данных из начала занятия.

In [ ]:
###########
# 
# YOUR CODE
# 1) make a regression data
# 1) make a linear model with degree = 20
# 2) Try lasso, ridge and elastic net regression with at least two or three different alphas
# 3) Draw the plot: on one axis (three plots in one) show the sparsity level for different lambdas of each regularizer
# 4) Draw the plot (or make table): show the error for different lambdas of each regularizer
#
###########

# Классификация
Конечно, для классификации также можно использовать регуляризацию. Рассмотрим логистическую регрессию как последовательность из линейной модели и сигмоиды. Напишите функцию для получения ошибки, как раньше, но используя новую модель.

In [ ]:
def sigmoid(z):
    return 1/(1+(np.exp((-z))))

from sklearn.metrics import log_loss

def get_error_plot_sigmoid(alphas, betas, norm_fn, gamma, x, y):
    result = pd.DataFrame(columns=["a", "b", "error"])
    for alpha in alphas:
        for beta in betas:
            res = {"a": alpha, "b": beta}
            model = LinearPredictionModel(coef=[[alpha, beta]], intercept=[[0]])
            res["error"] = mean_squared_error(y, sigmoid(model.predict(x))) + gamma * norm_fn(np.array([alpha, beta]))
            result = pd.concat([result, pd.DataFrame(res, index=[0])], axis=0, ignore_index=True)
    return result
            
            

In [ ]:
from sklearn.linear_model import LogisticRegression

x = np.random.randn(30,2)
real_alpha = 1.5
real_beta = 2.
line = real_alpha * x[:, 0] + real_beta * x[:, 1] 
y = (line + np.random.normal(loc=0, scale=0.5, size=x.shape[0]) > line).astype(int)
model = LogisticRegression().fit(x, y)
model.coef_
np.sum((model.predict(x) > 0.5) == y) / len(y)

Построим поверхность без нормализации. На самом деле, за счет того, что сигмоида - не выпуклая функция, поверхность также перестает быть выпуклой. 

In [ ]:
alphas = np.linspace( - 3, real_alpha + 3, 101)
betas = np.linspace( - 3, real_beta + 3, 101)
df_norm = get_error_plot_sigmoid(alphas, betas, l1_norm, 0.0, x, y)

fig = plt.figure(figsize=(12,8))
ax_1 = fig.add_subplot(121, projection='3d')
ax_2 = fig.add_subplot(122)
ax_1.plot_trisurf(df_norm["a"], df_norm["b"], df_norm["error"], alpha=0.5)
ax_2.tricontour(df_norm['a'], df_norm['b'], df_norm['error'], levels=15)
plt.show()

Вопрос: К чему это приводит?

Сравним также разные регрессоры для датасета с цифрами.

In [ ]:
random_indices = np.random.choice(X.index, 10000, replace=False)
X_rus = X.loc[random_indices]
y_rus = y_c.loc[random_indices]

X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.2, random_state=0)


In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(penalty=None, max_iter=30)
model.fit(X_train, y_train)
print("Train accuracy is: ", model.score(X_train / 255., y_train))
print("Test accuracy is: ", model.score(X_test / 255., y_test))

In [ ]:
sns.kdeplot(model.coef_.ravel())

In [ ]:
model = SGDClassifier(penalty="l2", alpha=0.001, max_iter=30)
model.fit(X_train, y_train)
print("Train accuracy is: ", model.score(X_train / 255, y_train))
print("Test accuracy is: ", model.score(X_test / 255, y_test))
sns.kdeplot(model.coef_.ravel())

In [ ]:
model = SGDClassifier(penalty="l1", alpha=0.1, max_iter=30)
model.fit(X_train, y_train)
print("Train accuracy is: ", model.score(X_train, y_train))
print("Test accuracy is: ", model.score(X_test, y_test))
sns.kdeplot(model.coef_.ravel())

In [ ]:
model = SGDClassifier(penalty="elasticnet", alpha=0.01, max_iter=30)
model.fit(X_train, y_train)
print("Train accuracy is: ", model.score(X_train, y_train))
print("Test accuracy is: ", model.score(X_test, y_test))
sns.kdeplot(model.coef_.ravel())

Рассмотрим, что будет, если использовать функцию квадратичной ошибки

In [ ]:
model = SGDClassifier(penalty="l1", alpha=0.01, max_iter=200, loss="squared_error")
model.fit(X_train, y_train)
print("Train accuracy is: ", model.score(X_train, y_train))
print("Test accuracy is: ", model.score(X_test, y_test))
sns.kdeplot(model.coef_.ravel())


## Дополнительная секция. Lars
Least angle regrssion (LARS) — это алгоритм регрессии для многомерных данных, разработанный Брэдли Эфроном, Тревором Хасти, Иэном Джонстоном и Робертом Тибширани (он же автор Lasso). LARS немного похож на PCA. На каждом этапе он находит признак, наиболее коррелирующий с целевой переменной. Когда существует несколько признаков, имеющих одинаковую корреляцию, вместо продолжения вдоль одного признака, процесс продолжается в направлении, равноугольном между ними.

- Он численно эффективен при p >> n.
- В вычислительном отношении он так же быстр, как и последовательный выбор признаков, и имеет тот же порядок сложности, что и обычный метод наименьших квадратов.
- Он создает полный кусочно-линейный путь решения, который полезен при перекрестной проверке или аналогичных попытках настройки модели.
- Если два признака почти одинаково коррелируют с целью, то их коэффициенты должны увеличиваться примерно с одинаковой скоростью. При этом алгоритм является более стабильным, чем прямой выбор признаков.
Однако, поскольку LARS основан на итеративной переработке ошибок, он может оказаться особенно чувствительным к влиянию шума. Эта проблема подробно обсуждается Вейсбергом в дискуссионном разделе книги Эфрона и др. (2004).


In [ ]:
data = load_diabetes()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
from sklearn.linear_model import lars_path, LassoLars, Lars
from sklearn import metrics
model = LassoLars(alpha=0.1)
model.fit(X_train, y_train)
print("Train r2 is: ", model.score(X_train, y_train))
print("Test r2  is: ", model.score(X_test, y_test))
sns.kdeplot(model.coef_.ravel())


In [ ]:
from sklearn.linear_model import lars_path, LassoLars, Lars
from sklearn import metrics
model = LassoLars(alpha=0.5)
model.fit(X_train, y_train)
print("Train r2 is: ", model.score(X_train, y_train))
print("Test r2  is: ", model.score(X_test, y_test))
sns.kdeplot(model.coef_.ravel())


А как понять, какие признаки в результате важны, а какие нет?

In [ ]:
np.nonzero(model.coef_)

Вопрос: а какие признаки отберут другие методы?

In [ ]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=0.5)
model.fit(X_train, y_train)
print("Train r2 is: ", model.score(X_train, y_train))
print("Test r2  is: ", model.score(X_test, y_test))
sns.kdeplot(model.coef_.ravel())


С помощью Lars можно также посмотреть на "пути" переменных при уменьшении степени регуляризации. Так можно выбрать, например, l1 регуляризацию с необходимым уровнем спарсности/точности

In [ ]:
print("Computing regularization path using the LARS ...")
_, _, coefs = lars_path(X_train, y_train, method="lar", verbose=True)
xx = np.sum(np.abs(coefs.T), axis=1)
xx /= xx[-1]

plt.plot(xx, coefs.T)
ymin, ymax = plt.ylim()
#plt.vlines(xx, ymin, ymax, linestyle="dashed")
plt.xlabel("|coef| / max|coef|")
plt.ylabel("Coefficients")
plt.title("LASSO Path")
plt.show()